# DEVELOPMENT DRAFT 
# Introduction to Facebook API for Researchers

This notebook was written by [John Simpson](mailto:john.simpson@computecanada.ca) and is meant to provide some simple, working examples for researchers who would like to collect information from Facebook.  This workbook uses a Python library called the [Facebook SDK](https://pypi.org/project/facebook-sdk-python/) to access the Facebook graph and collect information.  Much of this workbook is drawn from Study Tonight's [Working with Facebook Graph API](https://www.studytonight.com/network-programming-in-python/facebook-graph-api).  Official documentation for the Facebook SDK can be found [HERE](https://facebook-sdk.readthedocs.io/en/latest/index.html).  Official documentation for the Facebook API can be found [HERE](https://developers.facebook.com/docs/graph-api/reference).

This notebook assumes:

1. Basic familiarity with the Jupyter Notebook environment.
2. A functioning python environment on the system it is run in and that you have the authority to install software on it.
3. That you have a Facebook account [HERE](https://www.facebook.com/)
4. That you pay attention to the various notes and warnings around the cells.

I won't promise you any support but if you send me a note I'll help as I am able.

With all this said, let's get started by installing the TwitterAPI library.

In [1]:
!pip install facebook-sdk

With the Facebook SDK library installed on the system we load both it and the the JSON (javascript object notation) library for use throughout this workbook in the next block.

In [4]:
import facebook
import json

[Note that almost every piece of code in the remainder of this workbook assumes that the two cells above have been run and run successfully.  If you open the workbook and immediately try to run a cell other than this one first then it is likely that you will receive an error.  Simply run the cells above and try to run the cell you want to run again.  If you are receiving errors then two likely possibilities are an incorrect installation of Python or no network connection.]

## Authentication

Use of this workbook–and the Facebook API (Application Programming Interface) in general—requires a Facebook account with the developer features turned on.  Once you have a regular Facebook account setup the develope features can can be quickly turned on by creating an app as follows:

1. Go to [https://developers.facebook.com/](https://developers.facebook.com/) and choose "Get Started" from the top right corner.  (If you do not see "Get Started" then you have likely already created an app.  Choose, "My Apps" instead and then choosing the option to create a new app _or_ jumping to step ________ if you would like to use the app you already have.)
2. You will be asked to share some information and to perform a test to prove you are not a bot.  Do this.
3. You will be asked for a name for your app.  The demo version used to produce this notebook was called "Test App for Training".  You can call it anything you would like.
4. You will be asked for an email to be contacted through if Facebook every needs to raise a concern about your app.  Provide this.

When complete you will be put on a page for your app.  It will have a url in the address bar that looks like `https://developers.facebook.com/apps/321150513924422/add/` and has a bunch of products that can be added to the app listed as blocks in the middle of the page.  We are not interested in any of this.  At least not yet.  What we are interested in is an "Access Token" (basically a special number that will show we have permission to access the graph).  In order to be able to get this for a production app you will first need to give Facebook some additional information, specifically a URL for a privacy policy and a URL for a terms of service agreement (you can find these by clicking on `Settings` in the left-hand menu of the app screen and then `Basic`).  If you need help creating a privacy policy have a look at the free policy builder provided by [privacypolicies.com](https://www.privacypolicies.com/blog/facebook-apps-privacy-policy-url/) and if you need a place to put it consider [github.com](https://www.github.com).

We're not actually going to produce an app that anyone else is going to have access to at this point so we can avoid the need for these by creating a development clone of our app.  Do this by:

1. Clicking on `My Apps` on the menu at the top of the page.
2. Click the downward arrow in the top right of the block showing the app you'd like to create a development clone of and choosing `Create Test App`.  Note that you may need to hover your pointer over the block to see the arrow.
3. Clicking through the process.

With a development app in place we can proceed to get an access token for the Facebook Graph API by:

1. Going to [https://developers.facebook.com/tools/explorer/](https://developers.facebook.com/tools/explorer/).
2. Making sure that the _test_ version of the app has been selected from the dropdown menus under the blue `Generate Access Token` button on the right-hand side of the screen..
2. Clicking on the blue `Generate Access Token` button.
3. Accepting the window that declares what information will be collected.

This will place a temporary access token in the text field about the `Generate Access Token` button.  It will look something like this:                   
        
       EAAEE8YsF4fEBAOitY3RuiRyzEg1l25TNEdDQo5JOfT34PJbOZAs21UZChmyC5scAZBDhNzwSw679ZA4AtS5ry1ZAehfFMViBZCVvzIGxZAfppZClob7c8OZBsBRQGpJB7Wt9bVdSEPKnD8oIt7O1IITYgJEVTl9JdNL8UcZB4xFKZBJObVsv9C19MZCUa2ZABiiXlrvIbZBvgvcJWvr2ue1fWKAEC6Ni1WzJ7G5o2OY4NM3ZBolZCgZDZD

You should avoid sharing this (the one above is either broken or expired) and good programming practice would have you never put it in your notebook but rather load it from an outside file so no one ever sees it.  For the purpose of this training session though this is overkill.  If you ever want to revoke an access token that has been compromised just go back to the page where it was generated and generate a new one.

Copy your access token over the sample token in the block below and run the block.

In [6]:
token = "EAAEE8YsF4fEBAOitY3RuiRyzEg1l25TNEdDQo5JOfT34PJbOZAs21UZChmyC5scAZBDhNzwSw679ZA4AtS5ry1ZAehfFMViBZCVvzIGxZAfppZClob7c8OZBsBRQGpJB7Wt9bVdSEPKnD8oIt7O1IITYgJEVTl9JdNL8UcZB4xFKZBJObVsv9C19MZCUa2ZABiiXlrvIbZBvgvcJWvr2ue1fWKAEC6Ni1WzJ7G5o2OY4NM3ZBolZCgZDZD"
graph = facebook.GraphAPI(token)

In [5]:
#fields = ['first_name', 'location{location}','email','link']
profile = graph.get_object('me',fields='first_name,location,link,email')
#return desired fields
print(json.dumps(profile, indent=4))

{
    "first_name": "John",
    "id": "10158689448086303"
}


Note that while we asked for location and email that neither of these was returned.  Why?  We haven't provided the necessary privacy and terms of service information required for larger access.

In [9]:
page_name = input("Enter a page name: ")
# list of required fields
fields = ['id','name','about','likes','link','band_members']
fields = ','.join(fields)
page = graph.get_object(page_name, fields=fields)
print(json.dumps(page,indent=4))

Enter a page name: SesameStreet


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/simpson/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-5567e61f1462>", line 5, in <module>
    page = graph.get_object(page_name, fields=fields)
  File "/Users/simpson/anaconda3/lib/python3.7/site-packages/facebook/__init__.py", line 126, in get_object
    return self.request("{0}/{1}".format(self.version, id), args)
  File "/Users/simpson/anaconda3/lib/python3.7/site-packages/facebook/__init__.py", line 313, in request
    raise GraphAPIError(result)
facebook.GraphAPIError: (#100) Page Public Metadata Access requires either app secret proof or an app token

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/simpson/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_trace

GraphAPIError: (#100) Page Public Metadata Access requires either app secret proof or an app token